In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
from scipy.spatial.distance import cosine

In [2]:
data = []
with open('checkins.dat', 'r') as file:
    for line in file:
        data.append(line[:-1])

In [3]:
data.pop(1)
data = list(map(lambda x: re.split('\s+\|\s+|\|', x), data))
data = list(map(lambda x: list(map(lambda y: y.strip(), x)), data))
columns = data[0]
data = data[1:-2]
DF = pd.DataFrame(data, columns = columns)
DF = DF[DF.latitude != '']

In [4]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 1 to 1021964
Data columns (total 6 columns):
id            396634 non-null object
user_id       396634 non-null object
venue_id      396634 non-null object
latitude      396634 non-null object
longitude     396634 non-null object
created_at    396634 non-null object
dtypes: object(6)
memory usage: 21.2+ MB


In [5]:
DF.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [6]:
DF.shape

(396634, 6)

In [7]:
offices = np.array([[33.751277, -118.188740], 
                   [25.867736, -80.324116], 
                   [51.503016, -0.075479], 
                   [52.378894, 4.885084], 
                   [39.366487, 117.036146],
                   [-33.868457, 151.205134]])

In [8]:
DF_cords = DF.iloc[:,3:5]

In [9]:
clustering = MeanShift(bandwidth = 0.1).fit(DF_cords)

DF_cords['labels'] = clustering.labels_.copy()
DF_centers = pd.DataFrame(clustering.cluster_centers_[clustering.labels_])
DF_cords['centers_la'] = DF_centers.iloc[:, 0]
DF_cords['centers_lo'] = DF_centers.iloc[:, 1]

D:\Conda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Conda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Conda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [ ]:
#also there is DataFrame.isin

In [10]:
y = np.bincount(DF_cords.labels)
ii = np.nonzero(y)[0]
dropout = []
for i, j in zip(ii,y[ii]):
    if j <= 15:
        dropout.append(i)

In [11]:
DF_cords['test'] = DF_cords.labels.apply(lambda x: 0 if x in dropout else 1)
DF_cords = DF_cords[DF_cords.test != 0]
DF_cords.drop(['test'], axis = 1, inplace = True)

In [12]:
DF_cords_forMapcustomizer = DF_cords.iloc[:, 0:2]
with open('cords.txt', 'w') as file:
    for la, lo in zip(DF_cords_forMapcustomizer.iloc[:, 0], DF_cords_forMapcustomizer.iloc[:, 1]):
        file.write(f'{la} {lo}\n')

In [15]:
cos_cords = []
DF_cords = DF_cords.iloc[:, 3:5]
DF_cords.drop_duplicates(inplace = True)
for la, lo in zip(DF_cords.iloc[:,0], DF_cords.iloc[:,1]):
    for i in offices:
        lst_ = []
        lst_.append(cosine([i[0], i[1]], [la, lo]))
        lst_.append(i[0])
        lst_.append(i[1])
        lst_.append(la)
        lst_.append(lo)
        cos_cords.append(lst_)

In [16]:
final = pd.DataFrame(cos_cords)
final.sort_values(0, axis = 0).head(20)

,0,1,2,3,4
737,1.496716e-10,-33.868457,151.205134,-33.866146,151.207082
13147,8.103884e-10,25.867736,-80.324116,26.339806,-81.778697
4044,2.116338e-09,33.751277,-118.188740,32.725323,-114.624342
23640,3.734615e-09,33.751277,-118.188740,28.709143,-100.499521
11856,5.783020e-09,33.751277,-118.188740,32.978657,-115.530267
6042,7.614950e-09,33.751277,-118.188740,27.800583,-97.396381
21349,9.548929e-09,25.867736,-80.324116,39.143087,-121.604133
1695,1.084836e-08,52.378894,4.885084,52.372489,4.892268
132,1.160512e-08,33.751277,-118.188740,33.675235,-117.854455
2383,1.923498e-08,25.867736,-80.324116,25.846206,-80.311245


In [154]:
with open('answer.txt', 'w') as file:
    file.write('-33.866146 151.207082')